# Hemocyte Recruitment Analysis - Preprocessing 
This notebook takes abdomen and hemocyte segmented features extracted by QuPath and consolidates features from all samples into dataframes for downstream analysis. Further, sample names are mapped and XY coordinates of hemocytes are scaled relative to the abdomen centroid. 

## Import required packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from os import path
from pathlib import Path
import glob
sns.set_style('white')

## Set up input/output directories and experiment info

In [2]:

base_path = Path("/Users/sbandya/Desktop/hemocyte_recruitment_image_quantification/")
sample_mappings = pd.read_csv(base_path.joinpath("data","sample_manifests","sample_manifest_allbatches.tsv"),sep="\t",header=0)


# Paths to abdomen measurements and hemocyte detection measurements
am_path= base_path.joinpath('results','v2_allbatches','annotation_measurements')
dm_path= base_path.joinpath('results','v2_allbatches','detection_measurements')

outputs = base_path.joinpath('results','v2_allbatches')




## Create a dataframe of abdomen features

In [3]:

def get_sample_condition(imagename,sample_mapping):
    #print(imagename)
    condition = sample_mapping[sample_mapping['old'] == imagename]
    return condition['condition'].iloc[0]

all_annotation_measurement_files = glob.glob(path.join(am_path , "*.txt"))
am_list = []
for amf in all_annotation_measurement_files:
    df = pd.read_csv(amf, index_col=None, header=0, sep="\t")
    am_list.append(df)


abdomen_coordinates = pd.concat(am_list, axis=0, ignore_index=True)


## annotate each fly abdomen with experimental group info 
abdomen_coordinates['condition'] = abdomen_coordinates['Image'].apply(get_sample_condition,sample_mapping=sample_mappings)
abdomen_coordinates['condition'] = np.where(abdomen_coordinates['condition'] == 'C', 'Fed', abdomen_coordinates['condition'])
abdomen_coordinates['condition'] = np.where(abdomen_coordinates['condition'] == 'S', 'Starved', abdomen_coordinates['condition'])

abdomen_coordinates.columns = abdomen_coordinates.columns.str.replace('Centroid X px', 'Centroid_X')
abdomen_coordinates.columns = abdomen_coordinates.columns.str.replace('Centroid Y px', 'Centroid_Y')
print(abdomen_coordinates.shape)
abdomen_coordinates[:5]



(54, 16)


,Image,Object ID,Object type,Name,Classification,Parent,ROI,Centroid_X,Centroid_Y,Num Detections,Num Positive,Positive %,Num Positive per px^2,Area px^2,Perimeter px,condition
0,85E_5_merge.tif,0e6eb74e-3009-40b4-9d2a-4c30fbd61ff7,Annotation,NaN,NaN,Root object (Image),Polygon,794.42,513.02,10.0,10.0,100.0,0.0000,396063.1,2755.2,Starved
1,76B_1_merge.tif,df0d21e9-1623-45d0-a3e4-d693874f639b,Annotation,NaN,NaN,Root object (Image),Polygon,658.75,474.10,2.0,2.0,100.0,0.0000,442338.0,2627.3,Starved
2,89_2_merge.tif,88aae0b3-96a4-49ed-88e2-8f03911deb75,Annotation,NaN,NaN,Root object (Image),Polygon,2106.70,1444.90,54.0,54.0,100.0,0.0000,4112249.4,8048.2,Starved
3,43_9_merge.tif,f61438e1-1faa-4e5e-af5f-6495d0cadb05,Annotation,NaN,NaN,Root object (Image),Polygon,1835.30,1553.40,6.0,6.0,100.0,0.0000,7205317.5,11319.3,Fed
4,86D_2_merge.tif,9668f985-0608-400b-bbf1-a1a714693af4,Annotation,NaN,NaN,Root object (Image),Polygon,564.47,449.05,32.0,32.0,100.0,0.0001,411532.5,2743.0,Starved


## Map Fly and Section IDs for each abdomen image

In [4]:
# update sample names in the image column to new names in the "fly_section" format 
def map_samples(imageid, sample_mapping_lookup):
    try:
        fly_section_id = sample_mapping_lookup[sample_mapping_lookup['old']==imageid]['new'].iloc[0]
        fly_id = sample_mapping_lookup[sample_mapping_lookup['old']==imageid]['new'].iloc[0].split("_")[0]
        section_id = sample_mapping_lookup[sample_mapping_lookup['old']==imageid]['new'].iloc[0].split("_")[1]
    except: 
        fly_section_id = "notfound"
        fly_id = "none"
        section_id = "none"
    return(pd.Series([fly_section_id,fly_id,section_id]))

abdomen_coordinates[['fly_section_ID','fly_ID','section_ID']] = abdomen_coordinates.apply(lambda x: map_samples(x.Image, sample_mapping_lookup=sample_mappings), axis=1) 

abdomen_coordinates

,Image,Object ID,Object type,Name,Classification,Parent,ROI,Centroid_X,Centroid_Y,Num Detections,Num Positive,Positive %,Num Positive per px^2,Area px^2,Perimeter px,condition,fly_section_ID,fly_ID,section_ID
0,85E_5_merge.tif,0e6eb74e-3009-40b4-9d2a-4c30fbd61ff7,Annotation,NaN,NaN,Root object (Image),Polygon,794.42,513.02,10.0,10.0,100.0,0.0000,396063.1,2755.2,Starved,85_25,85,25
1,76B_1_merge.tif,df0d21e9-1623-45d0-a3e4-d693874f639b,Annotation,NaN,NaN,Root object (Image),Polygon,658.75,474.10,2.0,2.0,100.0,0.0000,442338.0,2627.3,Starved,76_7,76,7
2,89_2_merge.tif,88aae0b3-96a4-49ed-88e2-8f03911deb75,Annotation,NaN,NaN,Root object (Image),Polygon,2106.70,1444.90,54.0,54.0,100.0,0.0000,4112249.4,8048.2,Starved,89_2,89,2
3,43_9_merge.tif,f61438e1-1faa-4e5e-af5f-6495d0cadb05,Annotation,NaN,NaN,Root object (Image),Polygon,1835.30,1553.40,6.0,6.0,100.0,0.0000,7205317.5,11319.3,Fed,43_9,43,9
4,86D_2_merge.tif,9668f985-0608-400b-bbf1-a1a714693af4,Annotation,NaN,NaN,Root object (Image),Polygon,564.47,449.05,32.0,32.0,100.0,0.0001,411532.5,2743.0,Starved,86_17,86,17
5,88_6_merge.tif,bbc2342e-a114-4655-8e38-88c2e4dfb3f2,Annotation,NaN,NaN,Root object (Image),Polygon,2416.30,1607.30,48.0,48.0,100.0,0.0000,3631098.8,7421.2,Starved,88_6,88,6
6,85F_2_merge.tif,67ce6422-835c-4af0-9379-5db691eea913,Annotation,NaN,NaN,Root object (Image),Polygon,724.44,438.12,50.0,50.0,100.0,0.0001,418967.3,2822.2,Starved,85_27,85,27
7,89_4_merge.tif,41c82681-83a3-413b-bd09-8a9b9a955e9a,Annotation,NaN,NaN,Root object (Image),Polygon,2381.10,1615.80,33.0,33.0,100.0,0.0000,3433601.5,7435.4,Starved,89_4,89,4
8,90C_2_merge.tif,a540d13b-ee71-43e9-b216-0ac9cdedb9f0,Annotation,NaN,NaN,Root object (Image),Polygon,665.53,462.63,13.0,13.0,100.0,0.0000,560752.8,3051.9,Starved,90_12,90,12
9,71_14_merge.tif,4f133dc7-9b58-4eab-9ce7-c208227a35c4,Annotation,NaN,NaN,Root object (Image),Polygon,2066.20,1607.40,85.0,85.0,100.0,0.0000,5373334.0,9396.6,Fed,71_14,71,14


## Create a dataframe of hemocyte features from all samples - scale coordinates relative to abdomen centroid and map fly and section IDs 

In [6]:
from scipy.spatial import distance
import math 
sns.set_palette("pastel")

def zero_scale_x(image, x, polygon):
    polygon_x = abdomen_coordinates[abdomen_coordinates['Image']==image]['Centroid_X'].iloc[0] 
    zero_scaled_x = x-polygon_x
    return(zero_scaled_x)

def zero_scale_y(image, y, polygon):
   polygon_y = abdomen_coordinates[abdomen_coordinates['Image']==image]['Centroid_Y'].iloc[0] 
   zero_scaled_y = y-polygon_y
   return(zero_scaled_y)

def get_centroid_distance(image,x,y,polygon):
    polygon_x = abdomen_coordinates[abdomen_coordinates['Image']==image]['Centroid_X'].iloc[0]
    polygon_y = abdomen_coordinates[abdomen_coordinates['Image']==image]['Centroid_Y'].iloc[0]
    centroid_distance = distance.euclidean([polygon_x, polygon_y], [x,y])

    return centroid_distance
    

all_detection_measurement_files = glob.glob(path.join(dm_path , "*.txt"))
dm_list = []

for dm in all_detection_measurement_files:
    #print(dm)
    detection_measurements = pd.read_csv(dm,sep="\t",header=0)
    detection_measurements.columns = detection_measurements.columns.str.replace('Centroid X px', 'Centroid_X')
    detection_measurements.columns = detection_measurements.columns.str.replace('Centroid Y px', 'Centroid_Y')

    try:
        detection_measurements["centroid_x_zeroscaled"] = detection_measurements.apply(
            lambda x: zero_scale_x(x.Image, x.Centroid_X,polygon=abdomen_coordinates), axis=1) 
        detection_measurements["centroid_y_zeroscaled"] = detection_measurements.apply(
            lambda x: zero_scale_y(x.Image, x.Centroid_Y,polygon=abdomen_coordinates), axis=1)
        detection_measurements["centroid_distances"] = detection_measurements.apply(
            lambda x: get_centroid_distance(x.Image, x.Centroid_X,x.Centroid_Y,polygon=abdomen_coordinates), axis=1)
    except ValueError: ## No detections
        print("No detections: ",dm)

    dm_list.append(detection_measurements)
    
    
#print(dm_list)
hemocyte_coordinates = pd.concat(dm_list, axis=0, ignore_index=True)
hemocyte_coordinates[['fly_section_ID','fly_ID','section_ID']] = hemocyte_coordinates.apply(lambda x: map_samples(x.Image, sample_mapping_lookup=sample_mappings), axis=1) 

hemocyte_coordinates[:5]

No detections:  /Users/sbandya/Desktop/hemocyte_recruitment_image_quantification/results/v2_allbatches/detection_measurements/90C_3_merge.txt
No detections:  /Users/sbandya/Desktop/hemocyte_recruitment_image_quantification/results/v2_allbatches/detection_measurements/52_20_merge.txt
No detections:  /Users/sbandya/Desktop/hemocyte_recruitment_image_quantification/results/v2_allbatches/detection_measurements/72D_1_merge.txt
No detections:  /Users/sbandya/Desktop/hemocyte_recruitment_image_quantification/results/v2_allbatches/detection_measurements/81_12_merge.txt
No detections:  /Users/sbandya/Desktop/hemocyte_recruitment_image_quantification/results/v2_allbatches/detection_measurements/72C_4_merge.txt
No detections:  /Users/sbandya/Desktop/hemocyte_recruitment_image_quantification/results/v2_allbatches/detection_measurements/73D_1_merge.txt
No detections:  /Users/sbandya/Desktop/hemocyte_recruitment_image_quantification/results/v2_allbatches/detection_measurements/76B_4_merge.txt


,Image,Object ID,Object type,Name,Classification,Parent,ROI,Centroid_X,Centroid_Y,Nucleus: Area,...,Delaunay: Max distance,Delaunay: Min distance,Delaunay: Mean triangle area,Delaunay: Max triangle area,centroid_x_zeroscaled,centroid_y_zeroscaled,centroid_distances,fly_section_ID,fly_ID,section_ID
0,85E_5_merge.tif,12e7a293-518f-48a2-9997-86e7ebb4f4db,Cell,NaN,Positive,Annotation,Polygon,946.38,276.39,51.0,...,320.5022,14.4500,1419.7205,2190.7332,151.96,-236.63,281.221618,85_25,85,25
1,85E_5_merge.tif,ef8a57e9-c5aa-4412-a0fe-e0d84c182813,Cell,NaN,Positive,Annotation,Polygon,833.74,282.54,47.0,...,394.1084,104.2156,23575.9160,51931.4102,39.32,-230.48,233.809950,85_25,85,25
2,85E_5_merge.tif,26702476-65b6-49f6-9c30-e5c6efc88056,Cell,NaN,Positive,Annotation,Polygon,937.34,290.19,155.0,...,357.3977,14.4500,12327.0889,28321.2891,142.92,-222.83,264.725018,85_25,85,25
3,85E_5_merge.tif,93300ecb-0520-4fa1-85a5-365060c0a422,Cell,NaN,Positive,Annotation,Polygon,1137.90,534.07,36.0,...,320.5022,179.3147,15256.0117,28321.2891,343.48,21.05,344.124415,85_25,85,25
4,85E_5_merge.tif,6346e3dd-4555-46a0-b2cd-74b4bf13e181,Cell,NaN,Positive,Annotation,Polygon,659.69,538.74,46.0,...,350.7946,146.7560,28134.4902,51931.4102,-134.73,25.72,137.163010,85_25,85,25


## Write abdomen and hemocyte features to csv files for downstream analysis 

In [7]:
hemocyte_coordinates.to_csv(outputs.joinpath("hemocyte_coordinates_consolidated.csv"), sep="\t",index=False)
abdomen_coordinates.to_csv(outputs.joinpath("abdomen_coordinates_consolidated.csv"), sep="\t",index=False)